In [1]:
import pandas as pd
import folium
from ipywidgets import interact, SelectMultiple

# Load the dataset
file_path = 'aisdk-2024-12-30.csv'  # Replace with your dataset path
df = pd.read_csv(file_path)

# Step 1: Filter rows where Longitude is 0
df = df[df['Longitude'] != 0]

# Step 2: Filter Ship type to include Cargo, Tanker, and Undefined (length > 100 for Undefined)
df = df[
    (df['Ship type'].isin(['Cargo', 'Tanker'])) |
    ((df['Ship type'] == 'Undefined') & (df['Length'] > 100))
]

# Step 3: Remove duplicate rows where at the same Timestamp the same IMO number repeats
df = df.drop_duplicates(subset=['# Timestamp', 'IMO'])

# Step 4: Create a grouped list of unique MMSI numbers along with their final Timestamp, Destination, and ETA
grouped_data = (
    df.groupby('MMSI')
    .apply(lambda group: group.iloc[-1])  # Get the last row for each MMSI
    [['MMSI', '# Timestamp', 'Destination', 'ETA']]
    .reset_index(drop=True)
)

# Convert the grouped data to a list for the dropdown
mmsi_list = grouped_data.apply(
    lambda row: f"MMSI: {row['MMSI']} | Destination: {row['Destination']} | ETA: {row['ETA']}",
    axis=1
).to_list()

# Step 5: Function to plot a map of the journey based on the selected MMSI
def plot_map(selected_mmsi):
    # Extract MMSI numbers from the selected items
    selected_mmsi_numbers = [int(info.split()[1]) for info in selected_mmsi]
    
    # Filter the data for the selected MMSI numbers
    filtered_data = df[df['MMSI'].isin(selected_mmsi_numbers)]
    
    # Initialize a folium map centered on the first point in the filtered data
    if not filtered_data.empty:
        center_lat = filtered_data['Latitude'].iloc[0]
        center_lon = filtered_data['Longitude'].iloc[0]
    else:
        center_lat, center_lon = 0, 0  # Default center if no data is selected
    
    m = folium.Map(location=[center_lat, center_lon], zoom_start=5)
    
    # Define colors for each navigational status
    status_colors = {
        'At anchor': 'blue',
        'Constrained by her draught': 'purple',
        'Moored': 'green',
        'Under way sailing': 'orange',
        'Under way using engine': 'red',
        'Unknown value': 'gray'
    }
    
    # Shorten the path: Keep one dot per 1-minute window
    filtered_data['# Timestamp'] = pd.to_datetime(filtered_data['# Timestamp'])
    filtered_data = filtered_data.sort_values(by='# Timestamp')
    filtered_data = filtered_data.groupby(
        [filtered_data['# Timestamp'].dt.floor('T'), 'MMSI'], as_index=False
    ).first()
    
    # Add paths and markers
    for mmsi in selected_mmsi_numbers:
        mmsi_data = filtered_data[filtered_data['MMSI'] == mmsi]
        coordinates = mmsi_data[['Latitude', 'Longitude']].values.tolist()
        timestamps = mmsi_data['# Timestamp'].tolist()
        statuses = mmsi_data['Navigational status'].tolist()
        
        # Draw paths with different colors for navigational status
        for i, coord in enumerate(coordinates):
            folium.CircleMarker(
                location=coord,
                radius=5,
                color=status_colors.get(statuses[i], 'black'),  # Default to black for unknown statuses
                fill=True,
                fill_opacity=0.7,
                popup=(
                    f"MMSI: {mmsi}<br>Timestamp: {timestamps[i]}<br>"
                    f"Status: {statuses[i]}"
                ),
            ).add_to(m)
    
    return m

# Step 6: Create a dropdown menu to select MMSI
dropdown = SelectMultiple(
    options=mmsi_list,
    value=[mmsi_list[0]],  # Default selection
    description='MMSI:',
    layout={'width': '600px'}
)

# Step 7: Interactive function to display the map
def interactive_map(selected_mmsi):
    map_widget = plot_map(selected_mmsi)
    display(map_widget)

# Display the dropdown and map
interact(interactive_map, selected_mmsi=dropdown)

C:\Users\herox\AppData\Local\Temp\ipykernel_9012\3204804454.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.iloc[-1])  # Get the last row for each MMSI


interactive(children=(SelectMultiple(description='MMSI:', index=(0,), layout=Layout(width='600px'), options=('…

<function __main__.interactive_map(selected_mmsi)>